# Pasta treinamento

In [ ]:
from ..treinamento.recomendacaoService import ContentRecommender
import time

# Instância global do serviço de recomendação
recomendacao_service = ContentRecommender()
try:
    time.sleep(10)
    recomendacao_service.treinarSistema()
    print("Modelo treinado com sucesso")

except Exception as e:
    print(f"Erro ao treinar modelo {e}")

Esse código cria e inicializa a instância global do sistema de recomendação baseado em conteúdo. Primeiro, ele importa a classe ContentRecommender, que contém toda a lógica de processamento do modelo TF-IDF que usamos no sistema, em seguida, cria uma instância, que é usada pelo backend sempre que é necessário gerar recomendações, garantindo que o modelo não precise ser refeito a cada requisição, enquanto o try atrasa um pouco a para garantir que todo o banco de dados tenha sido carregado.

Depois disso, o método recomendacao_service.treinarSistema() é chamado para realizar o treinamento inicial do modelo, processando os dados das viagens e das preferências cadastradas no banco. Assim, esse módulo garante que, ao iniciar o backend, o modelo de recomendação esteja pré-treinado e pronto para responder às solicitações feitas pelas rotas de recomendação (/api/recomendacao/recomendar).

# recomendacaoService.py

In [ ]:
import numpy as np
import pandas as pd
from ..viagem.viagem_service import ViagemService
from ..usuario.user_service import UserService

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors


class ContentRecommender:
    def __init__(self):
        self.viagem_service = ViagemService()
        self._usuario_service = UserService()
        self._vectorizer = TfidfVectorizer()
        self.matriz_utilidade = None
        self.knn = None
        self.tfidf_viagens = None
        self.tfidf_usuarios = None
        self.viagens_ids = []
        self.usuarios_ids = []

    # ==============================
    # Cria documentos textuais
    # ==============================
    def criarDocumentoTextualViagens(self):
        lista_viagens = self.viagem_service.getViagens()  # lista de dicts
        lista_documentos_viagens = []
        self.viagens_ids = []

        for viagem in lista_viagens:
            documento = []
            viagem_id = viagem["Id"]
            self.viagens_ids.append(viagem_id)

            # adiciona campos básicos da viagem
            for campo in ['Nome', 'Descricao', 'Clima', 'Preco', 'Companhia']:
                valor = str(viagem.get(campo, "")).strip()
                if valor:
                    documento.append(valor)

            # adiciona gêneros no documento
            generos = self.viagem_service.getGeneros(viagem_id)
            for genero in generos:
                nome = genero["Nome"].strip()
                intensidade = int(genero.get("Intensidade", 1))
                documento.extend([nome] * intensidade)  # repete pelo peso

            # adiciona lazeres no documento
            lazeres = self.viagem_service.getLazeres(viagem_id)
            for lazer in lazeres:
                nome = lazer["Nome"].strip()
                qualidade = int(lazer.get("Qualidade", 1))
                documento.extend([nome] * qualidade)

            lista_documentos_viagens.append(" ".join(documento))

        return lista_documentos_viagens

    def criarDocumentoTextualUsuario(self, usuario_id):
        documento = []

        # Preferências do usuário
        preferencias = self._usuario_service.getPreferencias(usuario_id)
        if preferencias and len(preferencias) > 0:
            clima, preco, companhia = preferencias[0]
            documento.extend([str(clima), str(preco), str(companhia)])

        # adiciona gêneros ao documento
        generos = self._usuario_service.getGeneros(usuario_id)
        for genero in generos:
            nome = genero["Nome"].strip()
            peso = int(genero.get("Preferencia", 1))
            documento.extend([nome] * peso) # repete pelo peso

        # adiciona lazeres ao documento
        lazeres = self._usuario_service.getLazeres(usuario_id)
        for lazer in lazeres:
            nome = lazer["Nome"].strip()
            peso = int(lazer.get("Intensidade", 1))
            documento.extend([nome] * peso) 

        return " ".join(documento)

    def criarDocumentoTextualUsuarios(self):
        # Cria um documento tf-idf para todos os usuarios do banco
        lista_usuarios = self._usuario_service.getUsuarios()
        documentos = []
        self.usuarios_ids = []

        for usuario in lista_usuarios:
            usuario_id = usuario["Id"]
            self.usuarios_ids.append(usuario_id)
            documentos.append(self.criarDocumentoTextualUsuario(usuario_id))

        return documentos

    # ==============================
    # TF-IDF
    # ==============================
    def aplicarTfIdfGlobal(self):
        # FUnção responsavel por criar documentos tf-idf para viagens e usuarios
        documentos_viagens = self.criarDocumentoTextualViagens()
        documentos_usuarios = self.criarDocumentoTextualUsuarios()

        self.tfidf_viagens = self._vectorizer.fit_transform(documentos_viagens)
        self.tfidf_usuarios = self._vectorizer.transform(documentos_usuarios)

        return self.tfidf_usuarios, self.tfidf_viagens

    # ==============================
    # Matriz de utilidade
    # ==============================
    def gerarMatrizUtilidade(self):
        # Função responsavel por gerar matriz de utilidade
        lista_avaliacoes = self.viagem_service.getTodasAvaliacoes()
        
        if lista_avaliacoes:
            # Se a matriz ja existe no banco de dados, cria um dataframe de viagem com usuario 
            df = pd.DataFrame(lista_avaliacoes, columns=['UsuarioId', 'ViagemId', 'Avaliacao'])
            matriz = df.pivot_table(index='UsuarioId', columns='ViagemId', values='Avaliacao').fillna(0)
            return matriz

        # Simula se não houver avaliações
        if self.tfidf_viagens is None or self.tfidf_usuarios is None:
            self.aplicarTfIdfGlobal()

        # Utiliza aproximação por cosseno dos documentos para gerar notas
        sim_matrix = cosine_similarity(self.tfidf_usuarios, self.tfidf_viagens)
        notas_estimadas = np.clip(sim_matrix * 5, 1, 5)
        matriz = pd.DataFrame(notas_estimadas, index=self.usuarios_ids, columns=self.viagens_ids)

        # Salva no banco
        for u_index, usuario_id in enumerate(self.usuarios_ids):
            for v_index, viagem_id in enumerate(self.viagens_ids):
                nota = round(matriz.iloc[u_index, v_index], 2)
                if self.viagem_service.getViagem(viagem_id):
                    self.viagem_service.avaliar(viagem_id, usuario_id, nota)

        return matriz

    # ==============================
    # KNN
    # ==============================
    def treinarSistema(self):
        #Função responsavel por aplicar o KNN nos documentos e matriz de utilidade
        self.tfidf_viagens = self._vectorizer.fit_transform(self.criarDocumentoTextualViagens())
        self.tfidf_usuarios = self._vectorizer.transform(self.criarDocumentoTextualUsuarios())
        self.matriz_utilidade = self.gerarMatrizUtilidade()

        self.knn = NearestNeighbors(metric='cosine', algorithm='brute')
        self.knn.fit(self.tfidf_viagens)

    # ==============================
    # Recomendações
    # ==============================
    def recomendarParaUsuarioNovo(self, usuario_id, top_recomendacoes=6):
        #Função responsavel para recomendar as viagens personalizadas ao usuario
        if self.tfidf_viagens is None or self.matriz_utilidade is None:
            raise Exception("O sistema precisa ser treinado antes de recomendar.")

        documento_usuario = self.criarDocumentoTextualUsuario(usuario_id)
        tfidf_usuario = self._vectorizer.transform([documento_usuario])

        sim_conteudo = cosine_similarity(tfidf_usuario, self.tfidf_viagens).flatten()
        sim_colab = self.matriz_utilidade.mean(axis=0).values

        # Utiliza 70% dos dados com aproximação por cosseno e 30% com as medias dos valores da matriz de utilidade
        score_final = 0.7 * sim_conteudo + 0.3 * sim_colab

        # Realiza o sort das viagens mais similares do usuario
        indices_top = score_final.argsort()[::-1][:top_recomendacoes]

        # Para cada viagem obtem o score final e a media das avaliacoes e adiciona a lista
        recomendacoes = []
        for idx in indices_top:
            viagem_id = self.viagens_ids[idx]
            score = float(score_final[idx])
            viagem = self.viagem_service.getViagem(viagem_id)
            if viagem:
                v = viagem[0]   
                rating = self.viagem_service.getAvaliacao(viagem_id)
                recomendacoes.append({
                    "viagem_id": viagem_id,
                    "nome": v.get("Nome"),
                    "descricao": v.get("Descricao"),
                    "custo": v.get("Preco"),
                    "img": v.get("Imagem"),
                    "score": round(score, 3),
                    "rating": rating
                })

        return recomendacoes


Esse arquivo implementa a classe ContentRecommender, responsável por todo o funcionamento do modelo de recomendação baseado em conteúdo do sistema. Ela combina técnicas de TF-IDF e KNN e similaridade do cosseno para identificar viagens que mais se alinham ao perfil e às preferências de cada usuário.A classe utiliza duas fontes principais:

ViagemService → acessa informações sobre as viagens cadastradas, como clima, preço, companhia, gêneros e lazeres.

UserService → coleta as preferências, gêneros e lazeres associados a cada usuário.

O funcionamento segue quatro etapas principais. Iniciando com o formulário de cada usuário que reúne seus atributos, nele, esses termos são repetidos de acordo com sua intensidade ou preferência, criando um texto que expressa o perfil de forma ponderada. Após isso, os documentos são transformados em vetores numéricos por meio do TfidfVectorizer, que calcula a importância de cada palavra dentro do conjunto total. Em sequência, caso existam avaliações no banco de dados, elas são usadas para gerar uma tabela (usuário × viagem) com as notas atribuídas e se não houver, o sistema estima essas notas calculando a similaridade de cosseno entre usuários e viagens, gerando valores simulados entre 1 e 5.
Por fim, o modelo é então ajustado com o algoritmo KNN, que permite encontrar as viagens mais próximas do perfil de um usuário. Na recomendação, o sistema combina 70% da similaridade de conteúdo com 30% da média de avaliações para gerar um score final, retornando as viagens mais compatíveis.

# Pasta de usuário

In [ ]:
from Models.usuario.user import User
from Models.usuario.senha import Senha
from Models.usuario.email_user import Email
from Repository.usuario.user_repository import UserContainer


class UserService:
    """
    Camada de serviço responsável pela lógica de negócio relacionada ao usuário.
    Atua como intermediária entre o repositório (acesso ao banco de dados) e o controlador.
    """

    def __init__(self):
        self._container = UserContainer()

    # ------------------------- CADASTRO E LOGIN -------------------------

    def cadastrar(self, nome: str, email: str, senha: str) -> User:
        """
        Cadastra um novo usuário após validação dos dados.

        :param nome: Nome do usuário.
        :param email: E-mail do usuário.
        :param senha: Senha do usuário.
        :return: Instância de User criada e armazenada no banco.
        """
        try:
            user_aux = User()
            user_aux.criar_usuario(nome, email, senha)

            id_user = self._container.cadastrar(nome, email, senha)
            user = self._container.pesquisarUsuario(id_user)

            if not user:
                raise Exception("Não foi possível encontrar o usuário após o cadastro.")

            return user

        except ValueError as e:
            raise e
        except Exception as e:
            raise Exception(f"Erro interno do sistema ao cadastrar usuário: {e}")

    def login(self, email: str, senha: str) -> User:
        """
        Realiza o login do usuário validando o e-mail e senha informados.

        :param email: E-mail do usuário.
        :param senha: Senha do usuário.
        :return: Objeto User caso as credenciais sejam válidas.
        """
        try:
            email_aux = Email()
            senha_aux = Senha()

            email_aux.set(email)
            senha_aux.set(senha)

            user = self._container.login(email, senha)
            if not user:
                raise ValueError("Dados inválidos. Verifique e-mail e senha.")

            return user

        except ValueError as e:
            raise e
        except Exception as e:
            raise Exception(f"Erro interno do sistema durante o login: {e}")

    # ------------------------- CONSULTAS -------------------------

    def usuario_existe(self, id_user: int) -> User | None:
        """
        Verifica se o usuário com o ID especificado existe.

        :param id_user: ID do usuário.
        :return: Objeto User se encontrado, ou None caso contrário.
        """
        if not id_user:
            raise ValueError("Id de usuário não foi fornecido.")

        return self._container.pesquisarUsuario(id_user)

    def getUsuarios(self) -> list:
        """Retorna a lista de todos os usuários cadastrados."""
        return self._container.getUsuarios()

    def getLazeres(self, user_id: int) -> list:
        """Retorna os lazeres associados a um usuário."""
        return self._container.getLazeres(user_id)

    def getGeneros(self, user_id: int) -> list:
        """Retorna os gêneros preferidos de um usuário."""
        return self._container.getGeneros(user_id)

    def getPreferencias(self, user_id: int) -> list:
        """Retorna as preferências de viagem de um usuário."""
        return self._container.getPreferencias(user_id)

    # ------------------------- PREFERÊNCIAS -------------------------

    def cadastrarPreferencias(self, id_user: int, clima: str, preco: str, companhia: str):
        """
        Cadastra as preferências de viagem de um usuário, com validação de valores.

        :param id_user: ID do usuário.
        :param clima: Tipo de clima preferido (ex: 'quente', 'frio', ...).
        :param preco: Faixa de preço desejada (ex: 'econômico', 'alto', ...).
        :param companhia: Tipo de companhia preferida (ex: 'sozinho', 'amigos', ...).
        """
        if not self.usuario_existe(id_user):
            raise ValueError("Usuário não existe.")

        valores_validos = {
            'clima': ['quente', 'frio', 'temperado', 'tropical'],
            'preco': ['economico', 'medio', 'alto', 'luxo'],
            'companhia': ['sozinho', 'casal', 'familia', 'amigos']
        }

        if clima not in valores_validos['clima']:
            raise ValueError(f"Clima inválido: {clima}")

        if preco not in valores_validos['preco']:
            raise ValueError(f"Preço inválido: {preco}")

        if companhia not in valores_validos['companhia']:
            raise ValueError(f"Companhia inválida: {companhia}")

        self._container.cadastrarPreferencias(id_user, clima, preco, companhia)


Esse arquivo implementa a classe UserService, que representa a camada de serviço responsável por toda a lógica relacionada aos usuários, atuando como um intermediário entre o controlador (Controller).

A principal função dessa classe é validar e organizar as regras de negócio antes que qualquer ação seja executada no banco, sendo dividia em blocos principais:

Cadastro e Login: O método cadastrar() cria um novo usuário, validando nome, e-mail e senha por meio das classes de domínio (User, Email, Senha). Em seguida, envia as informações para o repositório e retorna o usuário cadastrado. Enquanto método login() verifica se o e-mail e a senha informados são válidos, e caso estejam corretos, retorna o objeto do usuário correspondente.

Consultas e verificações: A classe disponibiliza métodos como usuario_existe(), getUsuarios(), getGeneros(), getLazeres() e getPreferencias(), que buscam informações relacionadas ao perfil do usuário e suas preferências armazenadas no banco.

Gerenciamento de preferências: O método cadastrarPreferencias() permite que um usuário defina suas preferências de viagem e valida se cada valor está dentro das opções permitidas. Caso os dados sejam válidos, as preferências são enviadas ao repositório para serem salvas na tabela Preferencias.

# Pasta viagem

In [ ]:
from Repository.viagem.viagem_repository import ViagemContainer

class ViagemService:
    """
    Camada de serviço responsável pela lógica de negócios relacionada às viagens.
    Esta classe atua como intermediária entre o controlador (ou view) e o repositório (ViagemContainer),
    centralizando regras de negócio e garantindo separação de responsabilidades.
    """

    def __init__(self):
        # Instancia o container (repositório) responsável pela comunicação com o banco de dados
        self._container = ViagemContainer()

    # -------------------------------
    # CADASTROS DE PREFERÊNCIAS DO USUÁRIO
    # -------------------------------

    def cadastrarPreferencias(self, viagem_user: int, clima: str, preco: str, companhia: str):
        """
        Registra as preferências gerais do usuário (clima, preço e companhia) no banco.
        """
        return self._container.cadastrarPreferencia(viagem_user, clima, preco, companhia)

    def cadastrarGeneros(self, viagem_user: int, id_genero: int, intensidade: int):
        """
        Cadastra os gêneros de viagem preferidos pelo usuário com o respectivo nível de intensidade.
        """
        return self._container.cadastrarGenero(viagem_user, id_genero, intensidade)

    def cadastrarLazeres(self, viagem_user: int, id_lazer: int, intensidade: int):
        """
        Cadastra as atividades de lazer preferidas do usuário, indicando a intensidade de interesse.
        """
        return self._container.cadastrarLazeres(viagem_user, id_lazer, intensidade)

    # -------------------------------
    # AVALIAÇÕES E COMPRAS DE VIAGENS
    # -------------------------------

    def avaliar(self, id_viagem: int, id_usuario: int, pontuacao: int):
        """
        Atualiza a avaliação de uma viagem feita por um usuário.
        """
        self._container.avaliar(id_viagem, id_usuario, pontuacao)

    def comprar(self, id_viagem: int, id_usuario: int, pontuacao: int):
        """
        Registra uma compra de viagem feita por um usuário.
        A pontuação inicial é normalmente zero, já que ainda não houve avaliação.
        """
        self._container.comprar(id_viagem, id_usuario, pontuacao)

    def checarCompra(self, id_viagem: int, id_usuario: int):
        """
        Verifica se o usuário já comprou determinada viagem.
        Retorna True se já houver registro, False caso contrário.
        """
        self._container.checarCompra(id_viagem, id_usuario)

    # -------------------------------
    # CONSULTAS DE VIAGENS
    # -------------------------------

    def getViagens(self) -> list:
        """
        Retorna a lista de todas as viagens disponíveis.
        """
        return self._container.selecionarTodas()

    def getViagemUsuario(self, usuario_id: int) -> list:
        """
        Retorna as viagens que um determinado usuário avaliou (ou comprou).
        """
        return self._container.getViagemUsuario(usuario_id)

    def getViagem(self, viagem_id: int) -> list:
        """
        Retorna os detalhes de uma viagem específica a partir do seu ID.
        """
        return self._container.selecionar(viagem_id)

    # -------------------------------
    # CONSULTAS DE GÊNEROS E LAZERES ASSOCIADOS ÀS VIAGENS
    # -------------------------------

    def getLazeres(self, viagem_id: int) -> list:
        """
        Retorna as atividades de lazer relacionadas a uma viagem específica.
        """
        return self._container.getLazeres(viagem_id)

    def getGeneros(self, viagem_id: int) -> list:
        """
        Retorna os gêneros (ex: aventura, descanso, cultura) associados a uma viagem específica.
        """
        return self._container.getGeneros(viagem_id)

    # -------------------------------
    # CONSULTAS DE AVALIAÇÕES
    # -------------------------------

    def getTodasAvaliacoes(self) -> list:
        """
        Retorna todas as avaliações registradas de todas as viagens.
        Pode ser usada em sistemas de recomendação ou estatísticas.
        """
        return self._container.getTodasAvaliacoes()

    def getAvaliacao(self, id_viagem: int) -> float:
        """
        Calcula e retorna a média de avaliações de uma viagem específica.

        Retorna:
            float: Média das avaliações arredondada para 2 casas decimais.
                   Retorna 0.0 caso não existam avaliações.
        """
        avaliacoes = self._container.getAvaliacao(id_viagem)

        # Se não houver avaliações, retorna 0
        if not avaliacoes:
            return 0.0

        # Soma as notas de avaliação e calcula a média
        soma = 0
        for avaliacao in avaliacoes:
            soma += avaliacao.get("Avaliacao", 0)

        return round(soma / len(avaliacoes), 2)
